In [1]:
install.packages("dslabs")
library(dslabs)

package 'dslabs' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\asus\AppData\Local\Temp\RtmpkJUepj\downloaded_packages


Warning message:
"package 'dslabs' was built under R version 3.6.3"


In [3]:
mnist <- read_mnist()
ncol(mnist$train$images)

[1] 784

In [2]:
data(heights)

In [4]:
# maximize F-score
cutoff <- seq(61, 70)
F_1 <- map_dbl(cutoff, function(x){
  y_hat <- ifelse(train_set$height > x, "Male", "Female") %>% 
    factor(levels = levels(test_set$sex))
  F_meas(data = y_hat, reference = factor(train_set$sex))
})

data.frame(cutoff, F_1) %>% 
  ggplot(aes(cutoff, F_1)) + 
  geom_point() + 
  geom_line()

max(F_1)

best_cutoff <- cutoff[which.max(F_1)]
best_cutoff

y_hat <- ifelse(test_set$height > best_cutoff, "Male", "Female") %>% 
  factor(levels = levels(test_set$sex))
sensitivity(data = y_hat, reference = test_set$sex)
specificity(data = y_hat, reference = test_set$sex)

ERROR: Error in map_dbl(cutoff, function(x) {: could not find function "map_dbl"


In [5]:
p <- 0.9
n <- length(test_index)
y_hat <- sample(c("Male", "Female"), n, replace = TRUE, prob=c(p, 1-p)) %>% 
  factor(levels = levels(test_set$sex))
mean(y_hat == test_set$sex)

# ROC curve
probs <- seq(0, 1, length.out = 10)
guessing <- map_df(probs, function(p){
  y_hat <- 
    sample(c("Male", "Female"), n, replace = TRUE, prob=c(p, 1-p)) %>% 
    factor(levels = c("Female", "Male"))
  list(method = "Guessing",
       FPR = 1 - specificity(y_hat, test_set$sex),
       TPR = sensitivity(y_hat, test_set$sex))
})
guessing %>% qplot(FPR, TPR, data =., xlab = "1 - Specificity", ylab = "Sensitivity")

cutoffs <- c(50, seq(60, 75), 80)
height_cutoff <- map_df(cutoffs, function(x){
  y_hat <- ifelse(test_set$height > x, "Male", "Female") %>% 
    factor(levels = c("Female", "Male"))
   list(method = "Height cutoff",
        FPR = 1-specificity(y_hat, test_set$sex),
        TPR = sensitivity(y_hat, test_set$sex))
})

# plot both curves together
bind_rows(guessing, height_cutoff) %>%
  ggplot(aes(FPR, TPR, color = method)) +
  geom_line() +
  geom_point() +
  xlab("1 - Specificity") +
  ylab("Sensitivity")

library(ggrepel)
map_df(cutoffs, function(x){
  y_hat <- ifelse(test_set$height > x, "Male", "Female") %>% 
    factor(levels = c("Female", "Male"))
   list(method = "Height cutoff",
        cutoff = x, 
        FPR = 1-specificity(y_hat, test_set$sex),
        TPR = sensitivity(y_hat, test_set$sex))
}) %>%
  ggplot(aes(FPR, TPR, label = cutoff)) +
  geom_line() +
  geom_point() +
  geom_text_repel(nudge_x = 0.01, nudge_y = -0.01)

# plot precision against recall
guessing <- map_df(probs, function(p){
  y_hat <- sample(c("Male", "Female"), length(test_index), 
                  replace = TRUE, prob=c(p, 1-p)) %>% 
    factor(levels = c("Female", "Male"))
  list(method = "Guess",
    recall = sensitivity(y_hat, test_set$sex),
    precision = precision(y_hat, test_set$sex))
})

height_cutoff <- map_df(cutoffs, function(x){
  y_hat <- ifelse(test_set$height > x, "Male", "Female") %>% 
    factor(levels = c("Female", "Male"))
  list(method = "Height cutoff",
       recall = sensitivity(y_hat, test_set$sex),
    precision = precision(y_hat, test_set$sex))
})

bind_rows(guessing, height_cutoff) %>%
  ggplot(aes(recall, precision, color = method)) +
  geom_line() +
  geom_point()
guessing <- map_df(probs, function(p){
  y_hat <- sample(c("Male", "Female"), length(test_index), replace = TRUE, 
                  prob=c(p, 1-p)) %>% 
    factor(levels = c("Male", "Female"))
  list(method = "Guess",
    recall = sensitivity(y_hat, relevel(test_set$sex, "Male", "Female")),
    precision = precision(y_hat, relevel(test_set$sex, "Male", "Female")))
})

height_cutoff <- map_df(cutoffs, function(x){
  y_hat <- ifelse(test_set$height > x, "Male", "Female") %>% 
    factor(levels = c("Male", "Female"))
  list(method = "Height cutoff",
       recall = sensitivity(y_hat, relevel(test_set$sex, "Male", "Female")),
    precision = precision(y_hat, relevel(test_set$sex, "Male", "Female")))
})
bind_rows(guessing, height_cutoff) %>%
  ggplot(aes(recall, precision, color = method)) +
  geom_line() +
  geom_point()

ERROR: Error in eval(expr, envir, enclos): object 'test_index' not found


In [3]:
# The reported_heights and heights datasets were collected from three classes taught in the Departments of Computer Science
# and Biostatistics, as well as remotely through the Extension School. 
# The Biostatistics class was taught in 2016 along with an online version offered by the Extension School. 
# On 2016-01-25 at 8:15 AM, during one of the lectures, the instructors asked student to fill in the sex and height
# questionnaire that populated the reported_heights dataset. 
# The online students filled out the survey during the next few days, after the lecture was posted online. 
# We can use this insight to define a variable which we will call type, to denote the type of student, inclass or online.


library(dslabs)
install.packages("dplyr")
library(dplyr)
library(lubridate)
data(reported_heights)

dat <- mutate(reported_heights, date_time = ymd_hms(time_stamp)) %>%
  filter(date_time >= make_date(2016, 01, 25) & date_time < make_date(2016, 02, 1)) %>%
  mutate(type = ifelse(day(date_time) == 25 & hour(date_time) == 8 & between(minute(date_time), 15, 30), "inclass","online")) %>%
  select(sex, type)

y <- factor(dat$sex, c("Female", "Male"))
x <- dat$type



  There is a binary version available but the source version is later:
      binary source needs_compilation
dplyr  1.0.6  1.0.7              TRUE

  Binaries will be installed
package 'dplyr' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\asus\AppData\Local\Temp\RtmpkJUepj\downloaded_packages


Warning message:
"package 'dplyr' was built under R version 3.6.3"

Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



Attaching package: 'lubridate'


The following object is masked from 'package:base':

    date




In [4]:
# Q1
# The type column of dat indicates whether students took classes in person ("inclass") or online ("online"). 
# What proportion of the inclass group is female? What proportion of the online group is female?

dat %>% group_by(type) %>% summarize(prop_female = mean(sex == "Female"))

type,prop_female
<chr>,<dbl>
inclass,0.6666667
online,0.3783784


In [5]:
# Q2
# In the course videos, height cutoffs were used to predict sex. Instead of height, use the type variable to predict sex. 
# Assume that for each class type the students are either all male or all female, based on the most prevalent sex in each 
# class type you calculated in Q1. Report the accuracy of your prediction of sex based on type. 
# You do not need to split the data into training and test sets.

install.packages("purrr")
library(purrr)
cutoff <- seq(61, 70)
#F_1 <- map_dbl(cutoff, function(x){
#    y_hat <- ifelse(train_set$type > x, "Male", "Female") %>%
#    factor(levels = levels(test_set$sex))
#})

y_hat <- ifelse(x == "online", "Male", "Female") %>% 
      factor(levels = levels(y))
mean(y_hat==y)

package 'purrr' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'purrr'"
Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problem copying C:\Users\asus\anaconda3\envs\r-tutorial\Lib\R\library\00LOCK\purrr\libs\x64\purrr.dll to C:\Users\asus\anaconda3\envs\r-tutorial\Lib\R\library\purrr\libs\x64\purrr.dll: Permission denied"
Warning message:
"restored 'purrr'"



The downloaded binary packages are in
	C:\Users\asus\AppData\Local\Temp\RtmpkJUepj\downloaded_packages


[1] 0.6333333

In [6]:
# Q3
# Write a line of code using the table() function to show the confusion matrix between y_hat and y. 
# Use the exact format function(a, b) for your answer and do not name the columns and rows. 
# Your answer should have exactly one space. 

# my 1st answer which names col and row
table(predicted = y_hat, actual = y)
# or 2nd
table(y_hat, y)

         actual
predicted Female Male
   Female     26   13
   Male       42   69

        y
y_hat    Female Male
  Female     26   13
  Male       42   69

In [10]:
# DON'T DO IT AGAIN - ALMOST CRUSHED COMPUTER!!!
old.packages()

,Package,LibPath,Installed,Built,ReposVer,Repository
ade4,ade4,C:/Users/asus/anaconda3/envs/r-tutorial/Lib/R/library,1.7-16,3.6.3,1.7-17,https://cran.r-project.org/src/contrib
Amelia,Amelia,C:/Users/asus/anaconda3/envs/r-tutorial/Lib/R/library,1.7.6,3.6.3,1.8.0,https://cran.r-project.org/src/contrib
arrow,arrow,C:/Users/asus/anaconda3/envs/r-tutorial/Lib/R/library,4.0.0,3.6.3,4.0.1,https://cran.r-project.org/src/contrib
arules,arules,C:/Users/asus/anaconda3/envs/r-tutorial/Lib/R/library,1.6-7,3.6.3,1.6-8,https://cran.r-project.org/src/contrib
askpass,askpass,C:/Users/asus/anaconda3/envs/r-tutorial/Lib/R/library,1.0,3.6.0,1.1,https://cran.r-project.org/src/contrib
backports,backports,C:/Users/asus/anaconda3/envs/r-tutorial/Lib/R/library,1.1.4,3.6.0,1.2.1,https://cran.r-project.org/src/contrib
biclust,biclust,C:/Users/asus/anaconda3/envs/r-tutorial/Lib/R/library,2.0.2,3.6.3,2.0.3,https://cran.r-project.org/src/contrib
blavaan,blavaan,C:/Users/asus/anaconda3/envs/r-tutorial/Lib/R/library,0.3-15,3.6.3,0.3-16,https://cran.r-project.org/src/contrib
boot,boot,C:/Users/asus/anaconda3/envs/r-tutorial/Lib/R/library,1.3-20,3.6.0,1.3-28,https://cran.r-project.org/src/contrib
broom,broom,C:/Users/asus/anaconda3/envs/r-tutorial/Lib/R/library,0.5.2,3.6.0,0.7.8,https://cran.r-project.org/src/contrib


In [ ]:
# DON'T DO IT!!! COMPUTER CRASHED!!!
update.packages(ask=FALSE)

also installing the dependencies 'lmom', 'svd', 'fastmap', 'gargle', 'ids', 'httpcode', 'fauxpas', 'triebeard', 'rio', 'flexclust', 'additivityTests', 'lavaan', 'bayesplot', 'diagis', 'gld', 'gap', 'qgam', 'R62S3', 'set6', 'curry', 'Rdpack', 'trust', 'lpSolveAPI', 'rle', 'dreamerr', 'fracdiff', 'credentials', 'zip', 'DiceDesign', 'multicool', 'dcurver', 'RcppGSL', 'tiff', 'RItools', 'conquer', 'textshaping', 'DEoptimR', 'fastGHQuad', 'farver', 'segmented', 'qap', 'geometries', 'cachem', 'spatstat.sparse', 'tensor', 'spData', 'LearnBayes', 'R.cache', 'rematch2', 'googledrive', 'googlesheets4', 'crul', 'webmockr', 'urltools', 'car', 'laeken'




  There are binary versions available but the source versions are later:
                    binary     source needs_compilation
gargle               1.1.0      1.2.0             FALSE
rio                 0.5.26     0.5.27             FALSE
lavaan               0.6-8      0.6-9             FALSE
bayesplot            1.8.0      1.8.1             FALSE
Rdpack               2.1.1      2.1.2             FALSE
zip                  2.1.1      2.2.0              TRUE
multicool           0.1-11     0.1-12              TRUE
RcppGSL              0.3.8      0.3.9              TRUE
textshaping          0.3.4      0.3.5              TRUE
DEoptimR             1.0-8      1.0-9             FALSE
cachem               1.0.4      1.0.5              TRUE
spData               0.3.8     0.3.10             FALSE
googledrive          1.0.1      2.0.0             FALSE
car                 3.0-10     3.0-11             FALSE
ade4                1.7-16     1.7-17              TRUE
Amelia               1.7.6    

Warning message:
"packages 'dplyr', 'lubridate', 'purrr' are in use and will not be installed"

package 'lmom' successfully unpacked and MD5 sums checked
package 'svd' successfully unpacked and MD5 sums checked
package 'fastmap' successfully unpacked and MD5 sums checked
package 'ids' successfully unpacked and MD5 sums checked
package 'httpcode' successfully unpacked and MD5 sums checked
package 'fauxpas' successfully unpacked and MD5 sums checked
package 'triebeard' successfully unpacked and MD5 sums checked
package 'flexclust' successfully unpacked and MD5 sums checked
package 'additivityTests' successfully unpacked and MD5 sums checked
package 'diagis' successfully unpacked and MD5 sums checked
package 'gld' successfully unpacked and MD5 sums checked
package 'gap' successfully unpacked and MD5 sums checked
package 'qgam' successfully unpacked and MD5 sums checked
package 'R62S3' successfully unpacked and MD5 sums checked
package 'set6' successfully unpacked and MD5 sums checked
package 'curry' successfully unpacked and MD5 sums checked
package 'trust' successfully unpacked and

Warning message:
"cannot remove prior installation of package 'colorspace'"Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problem copying C:\Users\asus\anaconda3\envs\r-tutorial\Lib\R\library\00LOCK\colorspace\libs\x64\colorspace.dll to C:\Users\asus\anaconda3\envs\r-tutorial\Lib\R\library\colorspace\libs\x64\colorspace.dll: Permission denied"Warning message:
"restored 'colorspace'"

...

Warning message:
"cannot remove prior installation of package 'utf8'"Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problem copying C:\Users\asus\anaconda3\envs\r-tutorial\Lib\R\library\00LOCK\utf8\libs\x64\utf8.dll to C:\Users\asus\anaconda3\envs\r-tutorial\Lib\R\library\utf8\libs\x64\utf8.dll: Permission denied"Warning message:
"restored 'utf8'"

package 'uuid' successfully unpacked and MD5 sums checked


Warning message:
"cannot remove prior installation of package 'uuid'"Warning message in file.copy(savedcopy, lib, recursive = TRUE):
"problem copying C:\Users\asus\anaconda3\envs\r-tutorial\Lib\R\library\00LOCK\uuid\libs\x64\uuid.dll to C:\Users\asus\anaconda3\envs\r-tutorial\Lib\R\library\uuid\libs\x64\uuid.dll: Permission denied"Warning message:
"restored 'uuid'"

package 'vcr' successfully unpacked and MD5 sums checked
package 'vdiffr' successfully unpacked and MD5 sums checked
package 'VGAMextra' successfully unpacked and MD5 sums checked
package 'VIM' successfully unpacked and MD5 sums checked
package 'weights' successfully unpacked and MD5 sums checked
package 'whisker' successfully unpacked and MD5 sums checked
package 'wk' successfully unpacked and MD5 sums checked
package 'WRS2' successfully unpacked and MD5 sums checked
package 'xfun' successfully unpacked and MD5 sums checked
package 'xml2' successfully unpacked and MD5 sums checked
package 'xts' successfully unpacked and MD5 sums checked
package 'yaml' successfully unpacked and MD5 sums checked
package 'zoo' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\asus\AppData\Local\Temp\RtmpYBvMoO\downloaded_packages


installing the source packages 'gargle', 'rio', 'lavaan', 'bayesplot', 'Rdpack', 'DEoptimR', 'spData', 'googledrive', 'car', 'broom', 'broom.mixed', 'DHARMa', 'doBy', 'emmeans', 'formatR', 'ggfortify', 'GLMMadaptive', 'hms', 'LaplacesDemon', 'logger', 'mc2d', 'modeldata', 'pre', 'rmarkdown', 'semTools', 'slackr', 'styler', 'tinytest', 'tinytex'

Warning message in install.packages(update[instlib == l, "Package"], l, repos = repos, :
"installation of package 'gargle' had non-zero exit status"Warning message in install.packages(update[instlib == l, "Package"], l, repos = repos, :
"installation of package 'rio' had non-zero exit status"Warning message in install.packages(update[instlib == l, "Package"], l, repos = repos, :
"installation of package 'lavaan' had non-zero exit status"Warning message in install.packages(update[instlib == l, "Package"], l, repos = repos, :
"installation of package 'bayesplot' had non-zero exit status"Warning message in install.packages(update[instlib == l, "Pa

In [7]:
# Q4
# What is the sensitivity of this prediction? You can use the sensitivity() function from the caret package.

install.packages("caret",dep = TRUE)
install.packages("ggplot2")
install.packages("lattice")
install.packages("lava")
install.packages("purrr")

library(ggplot2)
library(lattice)
library(lava)
library(purrr)
library(caret)



  There is a binary version available but the source version is later:
      binary source needs_compilation
caret 6.0-86 6.0-88              TRUE

  Binaries will be installed
package 'caret' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\asus\AppData\Local\Temp\RtmpkJUepj\downloaded_packages

  There is a binary version available but the source version is later:
        binary source needs_compilation
ggplot2  3.3.3  3.3.5             FALSE



installing the source package 'ggplot2'




package 'lattice' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\asus\AppData\Local\Temp\RtmpkJUepj\downloaded_packages
package 'lava' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\asus\AppData\Local\Temp\RtmpkJUepj\downloaded_packages


Warning message:
"package 'purrr' is in use and will not be installed"
Warning message:
"package 'lattice' was built under R version 3.6.3"
Warning message:
"package 'lava' was built under R version 3.6.3"

Attaching package: 'lava'


The following object is masked from 'package:ggplot2':

    vars


The following object is masked from 'package:dplyr':

    vars


Warning message:
"package 'caret' was built under R version 3.6.3"

Attaching package: 'caret'


The following object is masked from 'package:purrr':

    lift




ERROR: Error in is.factor(reference): argument "reference" is missing, with no default


In [8]:
# Q4
sensitivity(y_hat, y)

[1] 0.3823529

In [9]:
# Q5 
# What is the specificity of this prediction? You can use the specificity() function from the caret package.

specificity(y_hat, y)

[1] 0.8414634

In [10]:
# Q6
# What is the prevalence (% of females) in the dat dataset defined above? 

prev <- mean(y == "Female")
prev

[1] 0.4533333